# Object detector



In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing libraries

In [ ]:
from __future__ import print_function, division
import cv2
import torchvision
import math
import os
import torch
import random
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image, ImageEnhance
import matplotlib.patches as patches
import time
from tqdm import tqdm
import torchvision.transforms.functional as TF
import warnings
import torchvision.ops as ops
warnings.filterwarnings('ignore')

<h3>Import datasets</h3>

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # set device

# Global parameters
num_classes = 3
batch_size = 4
learning_rate = 1e-4
num_of_epochs = 10

In [ ]:
# Load Data
class AnimalDataset(Dataset):
    def __init__(self, csv_file, root_dir=None, transform = None):
        self.ann = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.class2id = {
            "29": 1,
            "38": 2,
        }
    def __len__(self):
        return len(self.ann)
    def __getitem__(self, idx):
        img_path = self.root_dir + '/' + self.ann["id"][idx]

        img = np.array(Image.open(img_path))
        label = self.class2id[str(self.ann.loc[idx, 'labels'])]
        bbox = self.ann.loc[idx, 'xmin':'ymax'].tolist()
        sample = (img, {"boxes": bbox, "labels": label})
        if self.transform:
            sample = self.transform(sample)
        return sample
    def get_id(self, idx):
        return self.ann["id"][idx]

<h3>Data augmentation</h3>

In [ ]:
#Custom transforms

class Normalize(object):
    def __call__(self, sample):
        img, _ = sample
        img = img.astype(np.float32) / 255
        img -= np.min(img)
        img /= np.max(img)
        sample = (img, sample[1])
        return sample

class Rescale(object):
    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, sample):
        image, bbox = sample[0], sample[1]['boxes']

        h, w = image.shape[:2]
        new_h, new_w = int(self.output_size), int(self.output_size)

        img = cv2.resize(image,(new_h, new_w))

        #resizing bbox
        bh, bw = bbox[2]-bbox[0], bbox[3]-bbox[1]
        center = [int(bbox[0]+bw/2), int(bbox[1]+bh/2)]
        bbox = bbox - np.array(center+center)
        bbox = np.multiply(bbox , np.array([new_w / w, new_h / h, new_w / w, new_h / h]))
        center = np.multiply(np.array(center+center), np.array([new_w / w, new_h / h, new_w / w, new_h / h]))
        bbox = bbox + center


        return (img, {'boxes': bbox, 'labels': sample[1]['labels']})

class Random_flip(object):
    def __call__(self, sample):
        image, bbox = sample[0], sample[1]['boxes']
        h, w = image.shape[:2]

        #Random horizontal flip
        if (random.randint(0,1)):
            image = cv2.flip(image, 1)
            bbox[0] = w - bbox[0]
            bbox[2] = w - bbox[2]
            bbox[0],bbox[2] = bbox[2],bbox[0]

        return (image, {'boxes': bbox, 'labels': sample[1]['labels']})

class ColorJitter(object):
    def __call__(self, sample):
        img = Image.fromarray(np.uint8(sample[0]*255)).convert('RGB')

        rand_c = 0.9 + random.random()/2
        filter = ImageEnhance.Brightness(img)
        img = filter.enhance(rand_c)

        filter = ImageEnhance.Contrast(img)
        img = filter.enhance(rand_c)

        filter = ImageEnhance.Color(img)
        img = filter.enhance(rand_c)

        img = np.array(img, dtype='uint8')/255

        return (img, sample[1])

class ToTensor(object):
    def __init__(self, size):
        self.size = size

    def __call__(self, sample):
        img, bbox = sample[0], np.array(sample[1]['boxes'])

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        img = img.transpose((2, 0, 1))

        # to be safe
        if bbox[0]<0:
            bbox[0]=0
        if bbox[1]<0:
          bbox[1]=0
        if bbox[2]>self.size:
          bbox[2] = self.size
        if bbox[3]>self.size:
          bbox[3] = self.size
        if bbox[0]>bbox[2]:
          bbox[0],bbox[2] = bbox[2],bbox[0]
        if bbox[1]>bbox[3]:
            bbox[1],bbox[3] = bbox[3],bbox[1]

        return (torch.from_numpy(img).double(), {
            'boxes': torch.from_numpy(bbox).double(),#.to(device),
            'labels': torch.tensor(torch.from_numpy(np.array(sample[1]["labels"])), dtype = torch.long)#.to(device)
        })

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

def data_prep(data, type):
    images, targets = data
    if type=='train': #todo for some reason?>
        images = list(img.to(device) for img in images)
    elif type  =='test':
        images = list(img.float().to(device) for img in images)

    targets = [{k: v.float().unsqueeze(0).to(device) for k, v in t.items()} for t in targets]
    for t in targets:
        for k,v in t.items():
            if k=='labels':
                t[k] = torch.tensor(v.to(device), dtype=torch.long)

    return images, targets

In [ ]:
transform = transforms.Compose([Rescale(512), Normalize(), Random_flip(), ColorJitter(), ToTensor(512)])
print("Load dataset")


dataset_train = AnimalDataset(csv_file='global_labels_notbalanced.csv',
                                  root_dir='/content/drive/MyDrive/TrainingData/dataset_copy',
                                  transform=transform
                                  )


data_loader_train = DataLoader(
    dataset = dataset_train,
    batch_size = batch_size,
    shuffle = True,
    num_workers=4,
    drop_last=True,
    collate_fn=collate_fn,
)


Load dataset


In [ ]:
# Model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to(device)

#Input feature number for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


params = [p for p in model.parameters() if p.requires_grad]

#optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
optimizer  = torch.optim.Adam(model.parameters(), lr=learning_rate) #todo choose optimiser

# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
lr_scheduler = None

loss_hist = Averager()
model.train()

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
#Load checkpoint
#start_epoch = 2
checkpoint = torch.load('/content/drive/MyDrive/TrainingData/faster8.pth')

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch']
#loss = checkpoint['loss']
#model.load_state_dict(checkpoint)

model.to(device)
model.train()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

<h2>Faster RCNN training</h2>

In [ ]:
#----------------------------------------------TRAIN-----------------------------------------------------------------
#start_epoch = -1
itr = (start_epoch+1)*len(data_loader_train)

for epoch in range(start_epoch+1, num_of_epochs):
    loss_hist.reset()

    print("Epoch - {} Started".format(epoch))
    st = time.time()
    epoch_loss = []

    for data in data_loader_train:

        itr += 1
        images, targets = data_prep(data, type='train')

        model = model.double().to(device)
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        loss_hist.send(losses.item())

        optimizer.zero_grad()

        losses.backward()

        optimizer.step()

        print('Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Objectness loss: {:1.5f} | RPN Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(
                epoch, itr, float(loss_dict['loss_classifier'].item()), float(loss_dict['loss_box_reg']), float(loss_dict['loss_objectness']), float(loss_dict['loss_rpn_box_reg']), float(losses)))
        with open('/content/drive/MyDrive/TrainingData/fasterrcnn_loss.txt', 'a') as f:
            f.write('Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Objectness loss: {:1.5f} | RPN Regression loss: {:1.5f} | Running loss: {:1.5f}\n'.format(
                epoch, itr, float(loss_dict['loss_classifier'].item()), float(loss_dict['loss_box_reg']), float(loss_dict['loss_objectness']), float(loss_dict['loss_rpn_box_reg']), float(losses)))
        

    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    et = time.time()
    print("\n Total Time - {}\n".format(int(et - st)))
    with open('/content/drive/MyDrive/TrainingData/fasterrcnn_loss.txt', 'a') as f:
        f.write("Total Time - {}\n".format(int(et - st)))

    #print(f"Epoch #{epoch} loss: {loss_hist.value}")
    #checkpoint
    torch.save({
        'epoch':epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': losses}, '/content/drive/MyDrive/TrainingData/model'+str(epoch)+'epoch.pth')



Epoch - 9 Started
Epoch: 9 | Iteration: 31366 | Classification loss: 0.02071 | Regression loss: 0.01737 | Objectness loss: 0.00677 | RPN Regression loss: 0.00240 | Running loss: 0.04725
Epoch: 9 | Iteration: 31367 | Classification loss: 0.02176 | Regression loss: 0.04270 | Objectness loss: 0.00016 | RPN Regression loss: 0.00135 | Running loss: 0.06598
Epoch: 9 | Iteration: 31368 | Classification loss: 0.03034 | Regression loss: 0.02847 | Objectness loss: 0.00323 | RPN Regression loss: 0.00207 | Running loss: 0.06411
Epoch: 9 | Iteration: 31369 | Classification loss: 0.01212 | Regression loss: 0.01772 | Objectness loss: 0.00001 | RPN Regression loss: 0.00106 | Running loss: 0.03092
Epoch: 9 | Iteration: 31370 | Classification loss: 0.01924 | Regression loss: 0.01942 | Objectness loss: 0.00003 | RPN Regression loss: 0.00091 | Running loss: 0.03960
Epoch: 9 | Iteration: 31371 | Classification loss: 0.01159 | Regression loss: 0.01093 | Objectness loss: 0.00039 | RPN Regression loss: 0.0013

In [ ]:
torch.save({
        'epoch':epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': losses}, '/content/drive/MyDrive/TrainingData/model'+str(epoch)+'epoch34334.pth')

<h1>RetinaNet implementation</h1>

In [ ]:
#model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True, num_classes = num_classes).to(device)
backbone = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True)
model = torchvision.models.detection.RetinaNet(backbone.backbone, num_classes=num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
lr_scheduler = None



Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_coco-eeacb38b.pth


In [ ]:
#Load checkpoint
checkpoint = torch.load('/content/drive/MyDrive/TrainingData/model0epoch.pth')

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch']
loss = checkpoint['loss']

<h2>RetinaNet training</h2>

In [ ]:
start_epoch = -1
itr = (start_epoch+1)*len(data_loader_train)

for epoch in range(start_epoch+1, num_of_epochs):

    print("Epoch - {} Started".format(epoch))
    st = time.time()
    epoch_loss = []

    for data in data_loader_train:
        itr+=1
        images, targets = data_prep(data, type='train')

        
        model.double().to(device)

        # Forward
        classification_loss, regression_loss = model(images, targets).values()

        # Calculating Loss
        classification_loss = classification_loss.mean()
        regression_loss = regression_loss.mean()
        loss = classification_loss + regression_loss

        if bool(loss == 0):
            continue

        # Calculating Gradients
        loss.backward()

        # Gradient Clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
                
        # Updating Weights
        optimizer.step()

        #Epoch Loss
        epoch_loss.append(float(loss))

        print(
            'Epoch: {} | Iteration: {}/{} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(
                epoch, itr, len(data_loader_train),float(classification_loss.item()), float(regression_loss.item()), np.mean(epoch_loss)))
        
        with open('/content/drive/MyDrive/TrainingData/retina_loss_notbalanced.txt', 'a') as f:
            f.write('Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}\n'.format(
                epoch, itr, float(classification_loss.item()), float(regression_loss.item()), np.mean(epoch_loss)))

        del classification_loss
        del regression_loss
        
    et = time.time()
    print("\n Total Time - {}\n".format(int(et - st)))
    with open('/content/drive/MyDrive/TrainingData/retina_loss_notbalanced.txt', 'a') as f:
        f.write("Total Time - {}\n".format(int(et - st)))
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    #checkpoint
    torch.save({
        'epoch':epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss}, '/content/drive/MyDrive/TrainingData/retina'+str(epoch)+'epoch_notbalanced.pth')


Epoch - 0 Started
Epoch: 0 | Iteration: 1/2403 | Classification loss: 1.13975 | Regression loss: 0.70913 | Running loss: 1.84887
Epoch: 0 | Iteration: 2/2403 | Classification loss: 1.14067 | Regression loss: 0.64121 | Running loss: 1.81538
Epoch: 0 | Iteration: 3/2403 | Classification loss: 1.12919 | Regression loss: 0.71046 | Running loss: 1.82347
Epoch: 0 | Iteration: 4/2403 | Classification loss: 1.11151 | Regression loss: 0.64043 | Running loss: 1.80559
Epoch: 0 | Iteration: 5/2403 | Classification loss: 1.01135 | Regression loss: 0.82804 | Running loss: 1.81235
Epoch: 0 | Iteration: 6/2403 | Classification loss: 0.99092 | Regression loss: 0.76104 | Running loss: 1.80228
Epoch: 0 | Iteration: 7/2403 | Classification loss: 0.85820 | Regression loss: 0.69957 | Running loss: 1.76735
Epoch: 0 | Iteration: 8/2403 | Classification loss: 0.75296 | Regression loss: 0.63806 | Running loss: 1.72031
Epoch: 0 | Iteration: 9/2403 | Classification loss: 1.50601 | Regression loss: 0.70852 | Runni

In [ ]:
torch.save({
        'epoch':epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss}, '/content/drive/MyDrive/TrainingData/model'+str(epoch)+'epoch.pth')

In [ ]:
def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    """
    Calculates intersection over union
    Parameters:
        boxes_preds (tensor): Predictions of Bounding Boxes (BATCH_SIZE, 4)
        boxes_labels (tensor): Correct Labels of Boxes (BATCH_SIZE, 4)
        box_format (str): midpoint/corners, if boxes (x,y,w,h) or (x1,y1,x2,y2)
    Returns:
        tensor: Intersection over union for all examples
    """

    # Slicing idx:idx+1 in order to keep tensor dimensionality
    # Doing ... in indexing if there would be additional dimensions
    # Like for Yolo algorithm which would have (N, S, S, 4) in shape
    if box_format == "midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    elif box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    # Need clamp(0) in case they do not intersect, then we want intersection to be 0
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)
    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-6)

def nms(y,iou_threshold,threshold,box_format='corners'):
    bbx = []
    #assert type(boxes) == list
    bbox = y['boxes']
    labels = y['labels']
    scores = y['scores']

    for i in range(len(scores)):
        bbx.append([labels[i],scores[i],bbox[i]])
    #print("STOP--- len before", len(bbx))
    bbx = [box for box in bbx if box[1] > threshold]
    bbx = sorted(bbx, key=lambda x: x[1], reverse=True)
    bbox_after_nms = []
    #print(labels)
    while bbx:
        chosen = bbx.pop(0)
        for box in bbx:
            if box[0] != chosen[0]:
                pass
                # print(intersection_over_union(torch.tensor(
                #         chosen[2:][0]),
                #         torch.tensor(box[2:][0]),
                #         box_format = box_format))
        bbx = [box for box in bbx
                if #box[0] != chosen[0] or #dleted temporary
                    (intersection_over_union(torch.tensor(chosen[2:][0]),torch.tensor(box[2:][0]),box_format = box_format)
                    > iou_threshold)
                ] #not compare different classes

        bbox_after_nms.append(chosen)
    #here label, score, box
    #orig box, label, score

    y = {'labels':[],
          'score':[],
          'boxes':[]
          }

    for b in bbox_after_nms:
        # for el in b:
        #     el.cuda()
        y['labels'].append(b[0])
        y['score'].append(b[1])
        y['boxes'].append(b[2])
    #print("STOP--- len before", len(bbox_after_nms))
    for k,v in y.items():
        y[k] = torch.stack(v, dim=0)

    return y

def view(images, labels, k, std=1, mean=0):
    class2id = {
        "deer": 1,
        "boar": 2,
    }
    images = list(images)
    labels = list(labels)

    for i in range(k):
        label_counts = np.bincount(np.array(labels[i]['labels'].cpu()))

        #majority vote
        print(labels, "labels\n")
        print('PREDICTED LABEL: ',[k for k,v in class2id.items() if v == np.argmax(label_counts)][0])
        print('_______________')

        im = np.array(images[i].cpu()).transpose((1,2,0))
        im = np.array(std) * im + np.array(mean)
        im = np.clip(im, 0,1)
        #img_conc = np.concatenate((img_conc, im), axis = 1)
        fig, ax = plt.subplots()
        ax.imshow(im)
        l = labels[i]['boxes'].cpu().numpy()
        # l[:, 2] = l[:, 2] - l[:, 0]
        # l[:, 3] = l[:, 3] - l[:, 1]
        for j in range(len(l[:,0])):

            rect = patches.Rectangle((l[j][0], l[j][1]), l[j][2]-l[j][0],l[j][3]-l[j][1],
                                      linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
        plt.show()

    # cv2.imshow("Data vizualization",img_conc.astype(np.uint8))
    # cv2.waitKey(0)
        # for j in range(len(l)):
        #     ax.add_patch(patches.Rectangle((l[j][0], l[j][1]), l[j][2], l[j][3], linewidth=2, edgecolor='w',
        #                                    facecolor='none'))

def show_losses(vis): #todo implement
    def is_num(x):
        if x in ['1','2','3','4','5','6','7','8','9','0','.']:
            return True
        else:
            return False

    with open('losses.txt', 'r') as f:
        losses = [[],[],[],[]] #todo what are the losses?
        for line in f:
            idx = [i+len('tensor(') for i in range(len(line)) if line.startswith('tensor(', i)]
            for j,id in enumerate(idx):
                a = line[id]
                num = ''
                while is_num(a):
                    num+=str(a)
                    a = line[id+len(num)]
                losses[j].append(float(num))
    if vis:
        read_loss = np.array(losses)
        iter_num = range(len(read_loss[0, :]))
        # plt.scatter(iter_num, read_loss[0, :])
        # plt.scatter(iter_num, read_loss[1, :])
        # plt.scatter(iter_num, read_loss[2, :])
        rsum = read_loss[0, :]+read_loss[1, :]+read_loss[2, :]
        plt.scatter(iter_num, rsum)
        plt.show()

In [ ]:
writer.close()
!tensorboard --logdir=runs

2021-05-12 15:21:48.254488: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.4.1 at http://localhost:6006/ (Press CTRL+C to quit)
